In [109]:
def get_google_scholar_search_url(query_terms : list[str], query_params : dict | None = None):

    scholar_base_url = "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5"

    search_url = f"{scholar_base_url}&q={'+'.join(query_terms)}&btnG="

    return search_url

In [110]:
def get_proxies_list():
    with open(file='./proxyscrape_premium_http_proxies.txt', mode='r') as proxies_file:
        proxies = proxies_file.readlines()
    return proxies

In [111]:
import yaml
with open('./headers.yml') as f_headers:
    browser_header = yaml.safe_load(f_headers)

In [112]:
browser_header['Chrome']

{'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
 'cache-control': 'max-age=0',
 'sec-fetch-dest': 'document',
 'sec-fetch-mode': 'navigate',
 'sec-fetch-site': 'same-origin',
 'sec-fetch-user': '?1',
 'upgrade-insecure-requests': '1',
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

In [113]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome, ChromeOptions
import uuid
import os
from pathlib import Path
import requests
import pandas as pd
from random import choice
from typing import Iterator
import logging
import yaml
def get_dataframe_shuffled(df : pd.DataFrame, column : str | None = None) -> Iterator[pd.Series]:

    rows = df.sample(frac=1).iterrows()

    for row in rows :
        if column :
            yield row[1][column]
        else :
            yield row[1]

class ProxiesHandler():
    free_proxies_providers = ["https://free-proxy-list.net"]

    _retrieved_proxies      = None
    _proxies                = set()
    _used_proxies           = set()
    _not_working_proxies    = set()
    _browser_header         = None

    def __init__(self) -> None:
        self._mimic_header = None

    def get_proxies_elements(self):
        for proxies_provider in self.free_proxies_providers :
            resp = requests.get(proxies_provider)
            proxy_list = pd.read_html(resp.text)[0]
            proxy_list["url"]="http://" + proxy_list["IP Address"] + ":" + proxy_list["Port"].astype(str)
            self._retrieved_proxies = proxy_list[proxy_list["Https"]=="yes"]

    def build_good_proxies(self):
        test_url = "https://httpbin.org/ip"


        for proxy_row in get_dataframe_shuffled(df=self.proxies_retrived):
            proxy_url = proxy_row['url']
            try :
                proxies = {
                    'http'  : proxy_url,
                    'https' : proxy_url
                }
                resp = requests.get(url=test_url, headers=self.browser_header['Chrome'], proxies=proxies, timeout=2)
                print(f"proxy inscription : {proxy_url}")
                proxy_ip = f"{proxy_row['IP Address']}:{proxy_row['Port']}" if proxy_row['Port'] else proxy_row['IP Address']
                # proxy_ip = {proxy_row['IP Address']}
                self._proxies.add(proxy_ip)
                self.proxies_retrived.drop(index=self.proxies_retrived[self.proxies_retrived['url']==proxy_url].index)
            except Exception :
                print(f"proxy eviction : {proxy_url}")
                print(f"mimic_header = {self.mimic_header}")
                self.proxies_retrived.drop(index=self.proxies_retrived[self.proxies_retrived['url']==proxy_url].index)
            if len(self._proxies) >= 5 :
                break



    @property
    def mimic_header(self) -> dict[str, str]:
        if not self._mimic_header :
            resp = requests.get(
                url='https://headers.scrapeops.io/v1/browser-headers',
                params={
                    'api_key': "eba35a93-5dc6-4c01-971b-57b649ef7b6b",
                    'num_results': '1'}
                )
            self._mimic_header = resp.json()['result'][0]
        
        return self._mimic_header
    
    
    @property
    def proxies_retrived(self) -> pd.DataFrame:
        if not isinstance(self._retrieved_proxies, pd.DataFrame) :
            self.get_proxies_elements()
            # dataframe = pd.DataFrame({'IP Address' : get_proxies_list()})
            # dataframe['IP Address'] = dataframe['IP Address'].replace('\\n', "", regex=True)
            # dataframe['url'] = "http://" + dataframe
            # self._retrieved_proxies = dataframe
        
        if self._retrieved_proxies.empty:
            self.get_proxies_elements()
            # dataframe = pd.DataFrame({'IP Address' : get_proxies_list()})
            # dataframe['IP Address'] = dataframe['IP Address'].replace('\\n', "", regex=True)
            # dataframe['url'] = "http://" + dataframe
            # self._retrieved_proxies = dataframe
        
        return self._retrieved_proxies
    
    @property
    def proxies(self) -> set:
        if not self._proxies :
            self.build_good_proxies()
            # self._proxies = set(self.proxies_retrived['IP Address'])
        return self._proxies
    
    @property
    def proxy(self) -> str:
        return self.proxies.pop()
    
    @property
    def browser_header(self):
        if not self._browser_header :
            with open('./headers.yml') as f_headers:
                self._browser_header = yaml.safe_load(f_headers)
        return self._browser_header

class ScrapingDriver():
    AGENT_LIST = [
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
        "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:24.0) Gecko/20100101 Firefox/24.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/91.0.4472.114 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0"
        ]
    _proxies_handler = ProxiesHandler()
    
    def __init__(self) -> None:
        pass

    def get_driver(self, url : str) -> tuple[Chrome, Path]:
        # lg.info("Creation d'un moteur de scraping.")
        # service = Service(executable_path='/usr/bin/chromedriver')
        chrome_options = ChromeOptions()

        # temp_folder = Path(os.getenv("TEMP_EMPL"))
        # temp_folder.mkdir(parents=True, exist_ok=True)
        temp_folder = Path('./temp_' + str(uuid.uuid4()))
        temp_folder.mkdir(parents=True, exist_ok=True)

        prefs = {
            "download.default_directory" : str(temp_folder.absolute()),
            'download.prompt_for_download': False,
            'plugins.always_open_pdf_externally': True,
            "plugins.plugins_disabled" : "Chrome PDF Viewer",
        }

        # chrome_options.add_argument("--headless=new")
        chrome_options.add_argument('--disable-blink-features')
        chrome_options.add_argument("--incognito")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_argument(f'--proxy-server={self._proxies_handler.proxy}')
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)

        chrome_options.add_experimental_option('prefs', prefs)

        

        driver = Chrome(
            options=chrome_options,
            # service=service
            )
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": choice(self.AGENT_LIST)})
        driver.get(url=url)

        return driver, temp_folder

In [114]:
ph = ProxiesHandler()

In [115]:
# resp = requests.get(url='https://api.proxyscrape.com/v2/account/datacenter_shared/proxy-list?auth=6ff2g7v49p686gac8bte&type=getproxies&country[]=all&protocol=http&format=json&status=online')

In [116]:
# resp.content

In [117]:
from pathlib import Path
from typing import Iterable, Any
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4.element import Tag
import os
from copy import copy
from search_app.core.services.webscraping.random_wait import wait_rand
class DocumentRetriever():

    _scraping_driver = ScrapingDriver()

    def __init__(self, document_extract:Tag, webdriver_package : dict[str, Chrome | Path]) -> None:

        print("Récupération des données du document.")

        self._webdriver_package = webdriver_package
        self.document_extract = document_extract
        self.url = copy(self.driver.current_url)

        self.result_id = document_extract.attrs['data-cid']

    def get_files(self) -> tuple[bytes, str] | None:
        print("Récupération des fichiers disponnibles sur la page.")
        self.__get_pdf_file_if_exist()

        if self.pdf_file :
            self.ris_file = self.__get_bilio_info()
        
            if self.ris_file :
                # return {"pdf_file" : self.pdf_file, "ris_file" : self.ris_file}
                return self.pdf_file, self.ris_file
            else :
                return None
        else :
                return None
        
    @property
    def driver(self) -> Chrome:
        return self._webdriver_package['driver']
    
    @driver.setter
    def driver(self, value):
        self._webdriver_package.update({
            'driver' : value
        })

    @property
    def temp_folder(self) -> Path:
        return self._webdriver_package['folder']
    
    @temp_folder.setter
    def temp_folder(self, value):
        self._webdriver_package.update({
            'folder' : value
        })

    def rebuild_driver(self):
        print('reconstruction du moteur de scraping')
        self.temp_folder.rmdir()
        new_driver_package  = self._scraping_driver.get_driver(url = self.url)
        self.driver         = new_driver_package['driver']
        self.temp_folder    = new_driver_package['folder']


    def __get_bilio_info(self, *args, **kwargs) -> dict:

        print("Récupération des données bibliographiques.")

        try :
            self.driver.find_element(by=By.XPATH, value=f"//div[@data-cid='{self.result_id}']//a[@aria-controls='gs_cit']").click()
            wait_rand(size="medium")
            WebDriverWait(self.driver, 20.0).until(lambda d: self.driver.find_element(by=By.XPATH, value="//a[@class='gs_citi' and contains(., 'RefMan')]"))
        except NoSuchElementException as e :
            retry = kwargs.get('retry', 3)
            if retry :
                print(f"Rééssaie de telechargement des données bibliographiques : tentative restantes {retry}")
                retry += -1
                self.rebuild_driver()
                self.__get_bilio_info(retry = retry)
            else:
                e_text = f"Les données bibliogrqphiques n'ont pu être récupérées"
                print(e_text, exc_info=True)
                return None
        except Exception as e:
            e_text = f"Les données bibliogrqphiques n'ont pu être récupérées"
            print(e_text, exc_info=True)
            return None

        ris_citation = self.driver.find_element(by=By.XPATH, value="//a[@class='gs_citi' and contains(., 'RefMan')]")

        if not ris_citation:
            print('Le document ne possède par de références bibliographiques téléchargeables')
            self.driver.find_element(by=By.XPATH, value=f"//div[@class='gs_md_d gs_md_ds gs_ttzi gs_vis']//a[@id='gs_cit-x']").click()
            return None
        ris_citation.click()
        wait_rand(size="small")

        dwl_dir = self.temp_folder
        files_in_temp = [f for f in dwl_dir.iterdir() if f.is_file()]
        if files_in_temp :
            ris_file = files_in_temp[0]
            # ris_file_path = dwl_dir.joinpath(ris_file)
            with open(ris_file.absolute(), mode="r") as risfile:
                ris_content = risfile.read()
            os.remove(ris_file.absolute())

        else :
            e_text = f"Erreur de récupération des références bibliographiques, le téléchargement n'a pas eu lieu."
            print(e_text)
            raise ConnectionError(e_text)
        self.driver.find_element(by=By.XPATH, value=f"//div[@class='gs_md_d gs_md_ds gs_ttzi gs_vis']//a[@id='gs_cit-x']").click()
        wait_rand(size="small")
        
        if ris_content :
            return ris_content
        else :
            return None
    
    def __download_pdf_file(self, *args, **kwargs):

        print("Téléchargement du fichier PDF.")

        try :
            print("Suivi du lien de téléchargement.")
            self.driver.find_element(by=By.XPATH, value=f"//a[@data-clk-atid='{self.result_id}']").click()
            wait_rand(size='medium')
        except NoSuchElementException as e :
            retry = kwargs.get('retry', 3)
            if retry :
                print(f"Rééssaie de telechargement du fichier pdf : tentative restantes {retry}")
                retry += -1
                self.rebuild_driver()
                self.__download_pdf_file(retry = retry)
            else:
                e_text = "Erreur lors de la récupération du fichier PDF (ouverture du téléchargement)."
                print(e_text, exc_info=True)
                return None
        except Exception as e :
            e_text = "Erreur lors de la récupération du fichier PDF (ouverture du téléchargement)."
            print(e_text, exc_info=True)
            return None

        print("Recupération du fichier dans le dossier temporaire.")
        dwl_dir = self.temp_folder
        files_in_temp = [f for f in  dwl_dir.iterdir() if f.is_file()]
        if files_in_temp :
            pdf_file = files_in_temp[0]
            if pdf_file :
                # pdf_file = dwl_dir.joinpath(pdf_file)
                
                print("Lecture du fichier pdf.")
                with open(pdf_file.absolute(), mode="rb") as pdf_local_file:
                    pdf = pdf_local_file.read()
                print("Suppression du fichier pdf du dossier temporaire.")
                os.remove(pdf_file.absolute())
                if pdf :
                    return pdf
                else:
                    return None
        else :
            return None
    
    def __get_pdf_file_if_exist(self):
        
        print("Tentative de récupération du fichier PDF.")

        is_pdf = None

        print("Repérage du lien de téléchargement.")
        pdf_block = self.document_extract.find("a", attrs={'data-clk-atid' : self.result_id})
        if pdf_block :
            pdf_tag = pdf_block.find("span", attrs={'class' : 'gs_ctg2'})
            if pdf_tag :
                pdf_tag = pdf_tag.text
                if pdf_tag :
                    if 'pdf' in pdf_tag.lower() :
                        is_pdf = True

        if is_pdf :
            self.pdf_file = self.__download_pdf_file()
        else :
            print("Le lien de téléchagrement PDF est absent")
            self.pdf_file = None

In [118]:
from bs4 import BeautifulSoup
# from search_app.core.services.webscraping.drivers import ScrapingDriver

sd = ScrapingDriver()

def get_webdriver(url :str):
    driver, temp_folder = sd.get_driver(url=url)

    wedriver_package = {
        'driver' : driver,
        'folder' : temp_folder
    }
    return wedriver_package

def get_research_pages_on_gs(url : str, nb_pages:int) -> list[str]:

    webdriver_package = get_webdriver(url=url)

    results_page = BeautifulSoup(webdriver_package["driver"].execute_script("return document.documentElement.outerHTML;"), 'html.parser')

    navigator = results_page.find("div", attrs={'id' : "gs_n", 'role' : "navigation"})
    pages = navigator.find_all("a", limit=nb_pages)
    base_url = "https://scholar.google.com"

    pages_urls = [base_url + page.get("href") for page in pages]

    pages_urls[:0] = [webdriver_package["driver"].current_url]

    webdriver_package["folder"].rmdir()
    webdriver_package["driver"].delete_all_cookies()
    webdriver_package["driver"].quit()

    return pages_urls

In [119]:
from search_app.core.services.webscraping.google_scholar_scraping import DocumentRetriever

def parse_page_for_gs(url: str) -> list[tuple[bytes, str]]:
    webdriver_package = get_webdriver(url=url)

    results_page = BeautifulSoup(webdriver_package["driver"].execute_script("return document.documentElement.outerHTML;"), 'html.parser')
    results = results_page.find_all('div', attrs={'class' : "gs_r gs_or gs_scl"})
    
    files = list()
    for result in results :
            extractor = DocumentRetriever(document_extract=result, webdriver_package=webdriver_package)
            file = extractor.get_files()
            if file :

                files.append(file)

    webdriver_package["driver"].delete_all_cookies()
    webdriver_package["driver"].quit()
    webdriver_package["folder"].rmdir()
    
    return files

In [120]:
def get_result_files(results_list = list[list[tuple[bytes, str]]]):
    results_out = list()
    for results in results_list :
        results_out.extend(results)

    return results_out

def retrieve_files(pages : list[str]) -> list[tuple[bytes, str]]:

    pages_scraping_list = list()

    for page in pages :
        print(f"page : {page}")
        pages_scraping_list.append(parse_page_for_gs(page))
     
    result = get_result_files(pages_scraping_list)
    
    return result

In [121]:
search_url = get_google_scholar_search_url(query_terms=['vector', 'database'])
pages = get_research_pages_on_gs(search_url, 1)
result = retrieve_files(pages=pages)

/var/folders/yp/9gm2w_f53xj6vsx7dgtz95_c0000gn/T/ipykernel_20010/1898748862.py:37: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  proxy_list = pd.read_html(resp.text)[0]


proxy inscription : http://189.240.60.169:9090
proxy eviction : http://52.16.232.164:3128
mimic_header = {'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8', 'sec-ch-ua': 'Google Chrome;v="89", "Chromium";v="89", ";Not A Brand";v="99"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': 'macOS', 'sec-fetch-site': 'none', 'sec-fetch-mod': '', 'sec-fetch-user': '?1', 'accept-encoding': 'gzip, deflate', 'accept-language': 'en-US,en;q=0.5'}
proxy eviction : http://185.64.208.103:53281
mimic_header = {'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8', 'sec-ch-ua'

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.142)
Stacktrace:
0   chromedriver                        0x00000001013c24c8 chromedriver + 4302024
1   chromedriver                        0x00000001013bae10 chromedriver + 4271632
2   chromedriver                        0x0000000100fec19c chromedriver + 278940
3   chromedriver                        0x0000000100fc7474 chromedriver + 128116
4   chromedriver                        0x0000000101053394 chromedriver + 701332
5   chromedriver                        0x00000001010664e4 chromedriver + 779492
6   chromedriver                        0x0000000101023004 chromedriver + 503812
7   chromedriver                        0x00000001010239ec chromedriver + 506348
8   chromedriver                        0x000000010138a510 chromedriver + 4072720
9   chromedriver                        0x000000010138efbc chromedriver + 4091836
10  chromedriver  

In [ ]:
search_url

"https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=vector+database&btnG=''"